# Weight forecasting with linear models

## Setup

In [260]:
from sqlalchemy import create_engine
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt

from sklearn.linear_model import LinearRegression
# I tried these, they don't give an obvious improvement
# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.linear_model import RidgeCV
# from sklearn.linear_model import Ridge


**NOTES:**

- Model improvements:
   - pull in run/walk/other steps: I think this could give better predictive power
   - build a probabilistic generative model: use pyro to build a model that incorporates uncertainty in the parameters. This should let you run a forecast that shows uncertainty increasing over time (like hurricane forecasts). 
   - figure out a relatively straightforward way to model the dynamics of water/waste mass so you can subract it off of the 'true' body mass. I think that could be throwing off the predictability.
   
- Model improvements - backlog:
   - pull in breakfast/lunch/dinner/snacks calories: binging for dinner probably has more impact than spreading it out over the day. Perhaps just break it into night/day calories, i.e. day=snacks+breakfast+lunch, night=dinner. This could also give better predictive power.
  

## Load data

In [2]:
server_dir = '/Users/jamieinfinity/Dropbox/Projects/WorldLine/worldline-wgt/server/'
cfg_file = server_dir + 'config/api_params.cfg'
db_dir = server_dir + 'db/'
backups_dir = db_dir + 'backups/'
db_name = 'worldline'
db_ext = '.db'
db_file_name = db_dir + db_name + db_ext

In [3]:
engine = create_engine('sqlite:///'+db_file_name)
with engine.connect() as conn, conn.begin():
    db_df = pd.read_sql_table('fitness', conn, index_col='Date', parse_dates=['Date'])

In [4]:
data_full_df = db_df[db_df.index>'2015-09-15'].copy()

In [5]:
data_full_df.head()

,Weight,Steps,Calories,WeightImputed,StepsSmoothed3Days,StepsSmoothed5Days,StepsSmoothed7Days,WeightSmoothed3Days,WeightSmoothed5Days,WeightSmoothed7Days,CaloriesSmoothed3Days,CaloriesSmoothed5Days,CaloriesSmoothed7Days
Date,,,,,,,,,,,,,
2015-09-16,183.5,12452.0,1929.0,0,12119.550125,11578.010838,11248.425059,183.014710,182.368615,181.967130,2205.634440,2252.844437,2275.657057
2015-09-17,185.4,9778.0,2509.0,0,12215.818506,11636.574795,11281.521325,182.869244,182.317058,181.952406,2217.575748,2257.086685,2277.650084
2015-09-18,183.1,16295.0,1375.0,0,12264.860638,11670.837143,11306.095822,182.669254,182.243530,181.926757,2230.719689,2261.764490,2279.647312
2015-09-19,180.7,15560.0,2307.0,0,12261.919556,11680.409143,11321.774774,182.430944,182.151778,181.891158,2244.446683,2266.886837,2281.605084
2015-09-20,181.0,15370.0,2807.0,0,12204.591464,11666.076773,11328.514206,182.172382,182.046212,181.846759,2257.839882,2272.442877,2283.466909


## EDA - using gaussian kernal smoothed averages

In [7]:
data_df = data_full_df[['WeightSmoothed7Days', 'CaloriesSmoothed7Days', 'StepsSmoothed7Days']].copy()
data_df = data_df.rename({'WeightSmoothed7Days':'w', 'CaloriesSmoothed7Days':'c', 'StepsSmoothed7Days':'s'},axis=1)
data_df['date'] = data_df.index
data_df['w_prev'] = data_df.w.shift(7)
data_df['c_prev'] = data_df.c.shift(7)
data_df['s_prev'] = data_df.s.shift(7)
data_df.dropna(inplace=True)
data_df['dw'] = data_df['w']-data_df['w_prev']

In [8]:
data_df.tail()

,w,c,s,date,w_prev,c_prev,s_prev,dw
Date,,,,,,,,
2020-03-03,171.913969,1949.532766,15446.777076,2020-03-03,172.524063,1956.920210,15009.297565,-0.610094
2020-03-04,171.852463,1948.642723,15493.928125,2020-03-04,172.417511,1955.808485,15082.348630,-0.565048
2020-03-05,171.796679,1947.810640,15537.363743,2020-03-05,172.317136,1954.694490,15152.383810,-0.520457
2020-03-06,171.746223,1947.037399,15577.271703,2020-03-06,172.223256,1953.592654,15218.978978,-0.477033
2020-03-07,171.700682,1946.322528,15613.865254,2020-03-07,172.136037,1952.515794,15281.839295,-0.435355


In [158]:
alt.Chart(data_df).mark_circle(size=60).encode(
    alt.X('c',
        scale=alt.Scale(domain=(1600, 2800))
    ),    
    y='dw',
    color='s',
    tooltip=['date', 'w', 'c', 's']
).properties(
    width=500,
    height=300
).interactive()


alt.Chart(...)

## Simple linear regression - using gaussian kernal smoothed averages

### Train/test split

In [134]:
data_modeling = data_df[['w_prev','c_prev','s_prev','c','s','w']].copy()
data_train = data_modeling[data_modeling.index<'2019-11-01'].copy()
data_test = data_modeling[data_modeling.index>='2019-12-01'].copy()
data_test['date'] = data_test.index

features=['w_prev','c_prev','s_prev','c','s']
x_train = data_train[features].values
y_train = data_train[['w']].values
x_test = data_test[features].values
y_test = data_test[['w']].values

features2=['w_prev','c','s']
x_train2 = data_train[features2].values
y_train2 = data_train[['w']].values
x_test2 = data_test[features2].values
y_test2 = data_test[['w']].values

### Train the model

In [135]:
model = LinearRegression().fit(x_train, y_train)

In [136]:
model2 = LinearRegression().fit(x_train2, y_train2)

In [137]:
model2.score(x_train2, y_train2)

0.997956138568977

In [138]:
model2.score(x_test2, y_test2)

0.9928709707384727

### Model performance on test set

In [139]:
y_pred = model.predict(x_test)
y_pred2 = model2.predict(x_test2)
data_test['w_pred'] = [x[0] for x in y_pred]
data_test['w_pred2'] = [x[0] for x in y_pred2]

In [140]:
ys = (169, 181)
w=alt.Chart(data_test).mark_circle(size=60).encode(
    x='date',   
    y=alt.Y('w',
        scale=alt.Scale(domain=ys)
    ),      
    tooltip=['date', 'w', 'c', 's']
).properties(
    width=500,
    height=300
).interactive()
wp=alt.Chart(data_test).mark_point(size=60,opacity=0.3, color='orange').encode(
    x='date',   
    y=alt.Y('w_pred',
        scale=alt.Scale(domain=ys)
    ),      
    tooltip=['date', 'w', 'c', 's']
).interactive()
wp2=alt.Chart(data_test).mark_point(size=60,opacity=0.3, color='red').encode(
    x='date',   
    y=alt.Y('w_pred2',
        scale=alt.Scale(domain=ys)
    ),      
    tooltip=['date', 'w', 'c', 's']
).interactive()
w+wp2

alt.LayerChart(...)

### Weekly weight change vs calories

In [141]:
[c_w, c_c, c_s] = list(model2.coef_[0])
c_0 = model2.intercept_[0]

weight_prev = [160, 170, 180]
steps = [5000, 10000, 15000]
cals = list(range(1500, 3000, 10))

wgt = []
for wp in weight_prev:
    for s in steps:
        wgt_df = pd.DataFrame({'calories':cals})
        w = c_0 + c_w*wp + c_c*np.array(cals) + c_s*s
        wgt_df['delta_w'] = w-wp
        wgt_df['weight'] = wp
        wgt_df['steps'] = s
        wgt.append(wgt_df)
wgt_df = pd.concat(wgt, ignore_index=True)

In [142]:
wgt = wgt_df[wgt_df.steps==10000].copy()
alt.Chart(wgt).mark_circle(size=60).encode(
    alt.X('calories',
        scale=alt.Scale(domain=(1600, 2800))
    ),    
    y='delta_w',
    color='weight',
    tooltip=['calories', 'steps', 'weight', 'delta_w']
).properties(
    width=500,
    height=300
).interactive()

alt.Chart(...)

In [143]:
wgt = wgt_df[wgt_df.weight==170].copy()
alt.Chart(wgt).mark_circle(size=60).encode(
    alt.X('calories',
        scale=alt.Scale(domain=(1600, 2800))
    ),    
    y='delta_w',
    color='steps',
    tooltip=['calories', 'steps', 'weight', 'delta_w']
).properties(
    width=500,
    height=300
).interactive()

alt.Chart(...)

### Forecasting with actual (c,s) trajectory

In [144]:
period = '1w'
wgt_forecast = pd.DataFrame(data_test.w_prev.resample(period, label='right').last())
wgt_forecast['c'] = data_df.c.resample(period, label='right').last()
wgt_forecast['s'] = data_df.s.resample(period, label='right').last()
wgt_forecast['w'] = data_df.w.resample(period, label='right').last()

In [145]:
wgt_forecast.head()

,w_prev,c,s,w
Date,,,,
2019-12-01,179.254094,2158.077221,14947.333663,179.078320
2019-12-08,179.078320,2169.324729,14845.486905,178.911958
2019-12-15,178.911958,2232.263730,14272.400232,178.924519
2019-12-22,178.924519,2281.184305,13859.484493,179.022005
2019-12-29,179.022005,2257.131096,14103.677066,178.954520


In [156]:
w_fc = []
wp = 179.254
for dt in wgt_forecast.index:
    df=wgt_forecast.loc[dt]
    wp = c_0 + c_w*wp + c_c*df.c + c_s*df.s
    w_fc.append(wp)
wgt_forecast['w_fc'] = w_fc
wgt_forecast['date'] = wgt_forecast.index

In [157]:
ch1=alt.Chart(wgt_forecast).mark_circle(size=60).encode(
    x='date',  
    y=alt.Y('w',
        scale=alt.Scale(domain=(170, 180))
    ),  
    tooltip=['date', 'w', 'w_fc']
).properties(
    width=500,
    height=300
).interactive()
ch2=alt.Chart(wgt_forecast).mark_circle(size=60,color='red', opacity=0.3).encode(
    x='date',  
    y=alt.Y('w_fc',
        scale=alt.Scale(domain=(170, 180))
    ),  
    tooltip=['date', 'w', 'w_fc']
).properties(
    width=500,
    height=300
).interactive()
ch1+ch2

alt.LayerChart(...)

### Forecasting 'what-if' scenarios

## EDA - using backward-looking weekly rolling averages

In [17]:
data_df.head()

,w,c,s
Date,,,
2015-09-16,183.5,1929.0,12452.0
2015-09-17,185.4,2509.0,9778.0
2015-09-18,183.1,1375.0,16295.0
2015-09-19,180.7,2307.0,15560.0
2015-09-20,181.0,2807.0,15370.0


In [248]:
period='7D'
period_days=7
min_periods=5

data_df = data_full_df[['Weight', 'Calories', 'Steps']].copy().rename({'Weight':'w', 'Calories':'c', 'Steps':'s'},axis=1)
data_df = data_df.rolling(period, min_periods=min_periods).mean()
temp_df = pd.DataFrame(index=pd.date_range(start="2015-09-16",end="2020-03-07"))
data_df = pd.merge(temp_df, data_df, how='left', left_index=True, right_index=True)
data_df['date'] = data_df.index
data_df['w_prev'] = data_df.w.shift(period_days)
data_df['c_prev'] = data_df.c.shift(period_days)
data_df['s_prev'] = data_df.s.shift(period_days)
data_df.dropna(inplace=True)
data_df['dw'] = data_df['w']-data_df['w_prev']

In [249]:
data_df.tail()

,w,c,s,date,w_prev,c_prev,s_prev,dw
2020-03-03,171.342857,1921.285714,15926.142857,2020-03-03,172.957143,1995.000000,15401.285714,-1.614286
2020-03-04,170.942857,1872.142857,17183.285714,2020-03-04,172.628571,2049.285714,13790.142857,-1.685714
2020-03-05,170.728571,1877.714286,16729.857143,2020-03-05,172.500000,2033.714286,14033.142857,-1.771429
2020-03-06,170.614286,1892.428571,16526.000000,2020-03-06,172.400000,2006.571429,13725.714286,-1.785714
2020-03-07,170.642857,1919.000000,16671.857143,2020-03-07,172.257143,1965.000000,13658.857143,-1.614286


In [250]:
alt.Chart(data_df).mark_circle(size=60).encode(
    x=alt.X('c',
        scale=alt.Scale(domain=(1400, 3200))
    ), 
    y=alt.X('dw',
        scale=alt.Scale(domain=(-4, 3))
    ),    
    color='s',
    tooltip=['date', 'w', 'c', 's']
).properties(
    width=500,
    height=300
).interactive()


alt.Chart(...)

## Simple linear regression - using backward-looking weekly rolling averages

### Train/test split

In [251]:
data_modeling = data_df[['w_prev','c_prev','s_prev','c','s','w']].copy()
data_train = data_modeling[data_modeling.index<'2019-11-01'].copy()
data_test = data_modeling[data_modeling.index>='2019-12-01'].copy()
data_test['date'] = data_test.index

features=['w_prev','c_prev','s_prev','c','s']
x_train = data_train[features].values
y_train = data_train[['w']].values
x_test = data_test[features].values
y_test = data_test[['w']].values

features2=['w_prev','c','s']
x_train2 = data_train[features2].values
y_train2 = data_train[['w']].values
x_test2 = data_test[features2].values
y_test2 = data_test[['w']].values

### Train the model

In [252]:
model = LinearRegression().fit(x_train, y_train)

In [253]:
model2 = LinearRegression().fit(x_train2, y_train2)
# model2 = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1]).fit(x_train2, y_train2)
# model2 = Ridge(alpha=0.01).fit(x_train2, y_train2)
# model2 = GradientBoostingRegressor(loss='lad', n_estimators=200).fit(x_train2, y_train2.reshape(len(y_train2)))

In [254]:
model2.score(x_train2, y_train2)

0.9843492508150941

In [255]:
model.score(x_test, y_test)

0.9682086544354604

In [256]:
model2.score(x_test2, y_test2)

0.9684302240016265

### Model performance on test set

In [187]:
y_pred = model.predict(x_test)
y_pred2 = model2.predict(x_test2)
data_test['w_pred'] = [x[0] for x in y_pred]
data_test['w_pred2'] = [x[0] for x in y_pred2]

In [189]:
ys = (169, 181)
w=alt.Chart(data_test).mark_circle(size=60, opacity=0.4, color='black').encode(
    x='date',   
    y=alt.Y('w',
        scale=alt.Scale(domain=ys)
    ),      
    tooltip=['date', 'w', 'c', 's']
).properties(
    width=500,
    height=300
).interactive()
wp=alt.Chart(data_test).mark_point(size=60,opacity=0.5, color='blue').encode(
    x='date',   
    y=alt.Y('w_pred',
        scale=alt.Scale(domain=ys)
    ),      
    tooltip=['date', 'w', 'c', 's']
).interactive()
wp2=alt.Chart(data_test).mark_point(size=60,opacity=0.5, color='red').encode(
    x='date',   
    y=alt.Y('w_pred2',
        scale=alt.Scale(domain=ys)
    ),      
    tooltip=['date', 'w', 'c', 's']
).interactive()
wp2+w

alt.LayerChart(...)

### Weekly weight change vs calories

In [190]:
[c_w, c_c, c_s] = list(model2.coef_[0])
c_0 = model2.intercept_[0]

weight_prev = [160, 170, 180]
steps = [5000, 10000, 15000]
cals = list(range(1500, 3000, 10))

wgt = []
for wp in weight_prev:
    for s in steps:
        wgt_df = pd.DataFrame({'calories':cals})
        w = c_0 + c_w*wp + c_c*np.array(cals) + c_s*s
        wgt_df['delta_w'] = w-wp
        wgt_df['weight'] = wp
        wgt_df['steps'] = s
        wgt.append(wgt_df)
wgt_df = pd.concat(wgt, ignore_index=True)

In [191]:
wgt = wgt_df[wgt_df.steps==10000].copy()
alt.Chart(wgt).mark_circle(size=60).encode(
    alt.X('calories',
        scale=alt.Scale(domain=(1600, 2800))
    ),    
    y='delta_w',
    color='weight',
    tooltip=['calories', 'steps', 'weight', 'delta_w']
).properties(
    width=500,
    height=300
).interactive()

alt.Chart(...)

In [192]:
wgt = wgt_df[wgt_df.weight==170].copy()
alt.Chart(wgt).mark_circle(size=60).encode(
    alt.X('calories',
        scale=alt.Scale(domain=(1600, 2800))
    ),    
    y='delta_w',
    color='steps',
    tooltip=['calories', 'steps', 'weight', 'delta_w']
).properties(
    width=500,
    height=300
).interactive()

alt.Chart(...)

### Forecasting with actual (c,s) trajectory

In [193]:
period = '1w'
wgt_forecast = pd.DataFrame(data_test.w_prev.resample(period, label='right').last())
wgt_forecast['c'] = data_df.c.resample(period, label='right').last()
wgt_forecast['s'] = data_df.s.resample(period, label='right').last()
wgt_forecast['w'] = data_df.w.resample(period, label='right').last()

In [195]:
wgt_forecast.head()

,w_prev,c,s,w
2019-12-01,179.657143,2122.857143,13199.571429,179.400000
2019-12-08,179.400000,2016.857143,17549.857143,178.514286
2019-12-15,178.514286,2156.571429,13687.285714,178.900000
2019-12-22,178.900000,2271.857143,14956.714286,178.800000
2019-12-29,178.800000,2562.428571,11207.571429,179.211018


In [196]:
w_fc = []
wp = 179.657
for dt in wgt_forecast.index:
    df=wgt_forecast.loc[dt]
    wp = c_0 + c_w*wp + c_c*df.c + c_s*df.s
    w_fc.append(wp)
wgt_forecast['w_fc'] = w_fc
wgt_forecast['date'] = wgt_forecast.index

In [197]:
ch1=alt.Chart(wgt_forecast).mark_circle(size=60).encode(
    x='date',  
    y=alt.Y('w',
        scale=alt.Scale(domain=(170, 180))
    ),  
    tooltip=['date', 'w', 'w_fc', 'c', 's']
).properties(
    width=500,
    height=300
).interactive()
ch2=alt.Chart(wgt_forecast).mark_circle(size=60,color='red', opacity=0.3).encode(
    x='date',  
    y=alt.Y('w_fc',
        scale=alt.Scale(domain=(170, 180))
    ),  
    tooltip=['date', 'w', 'w_fc', 'c', 's']
).properties(
    width=500,
    height=300
).interactive()
ch1+ch2

alt.LayerChart(...)

### Forecasting with actual (c,s) trajectory

In [224]:
wgt_forecast = data_test[['date','w_prev','w','c','s']].copy()

wprev = list(wgt_forecast.w_prev.iloc[0:7].values)
w_fc = []

index=0
for dt in wgt_forecast.index:
    df = wgt_forecast.loc[dt]
    w = c_0 + c_w*wprev[index] + c_c*df.c + c_s*df.s
    w_fc.append(w)
    wprev.append(w)
    index+=1
wgt_forecast['w_fc'] = w_fc
wgt_forecast['wprev_fc'] = wprev[7:]

In [226]:
ch1=alt.Chart(wgt_forecast).mark_circle(size=60).encode(
    x='date',  
    y=alt.Y('w',
        scale=alt.Scale(domain=(170, 180))
    ),  
    tooltip=['date', 'w', 'w_fc', 'c', 's', 'w_prev']
).properties(
    width=500,
    height=300
).interactive()
ch2=alt.Chart(wgt_forecast).mark_circle(size=60,color='red', opacity=0.3).encode(
    x='date',  
    y=alt.Y('w_fc',
        scale=alt.Scale(domain=(170, 180))
    ),  
    tooltip=['date', 'w', 'w_fc', 'c', 's', 'w_prev']
).properties(
    width=500,
    height=300
).interactive()
ch1+ch2

alt.LayerChart(...)

### Forecasting 'what-if' scenarios

In [317]:
model2 = LinearRegression().fit(x_train2, y_train2)
[c_w, c_c, c_s] = list(model2.coef_[0])
c_0 = model2.intercept_[0]

def weight_pred(wgt_prev, cals, steps):
    return c_0 + c_w*wgt_prev + c_c*cals + c_s*steps
    
def make_forecast(wgt_prev, cals, steps, date_initial, weeks):
    d = datetime.datetime.strptime(date_initial, '%Y-%m-%d').date()
    wp = wgt_prev
    dates = []
    wgts = []
    for i in range(weeks):
        wp = weight_pred(wp, cals, steps)
        wgts.append(wp)
        dates.append(d)
        d = d + datetime.timedelta(days=7)
    res = pd.DataFrame({'date':dates, 'weight':wgts})
    res['date']=pd.to_datetime(res['date'])
    res['cals'] = cals
    res['steps'] = steps
    res['label'] = str(cals)+'|'+str(steps)
    return res

def make_multiple_forecasts(model_scenarios, date_initial, weeks):
    res = []
    for params in model_scenarios:
        wgt = make_forecast(params['wgt_prev'], params['cals'], params['steps'], date_initial, weeks)
        res.append(wgt)
    res = pd.concat(res, ignore_index=True)
    return res

In [369]:
d0 = '2020-01-05'
w_prev = 180
params = [
    {'wgt_prev':w_prev, 'cals':1900, 'steps':12000},
    {'wgt_prev':w_prev, 'cals':2000, 'steps':12000},
    {'wgt_prev':w_prev, 'cals':1800, 'steps':12000},
    {'wgt_prev':w_prev, 'cals':1900, 'steps':14000},
    {'wgt_prev':w_prev, 'cals':2000, 'steps':14000},
    {'wgt_prev':w_prev, 'cals':1800, 'steps':14000},
    {'wgt_prev':w_prev, 'cals':1900, 'steps':16000},
    {'wgt_prev':w_prev, 'cals':2000, 'steps':16000},
    {'wgt_prev':w_prev, 'cals':1800, 'steps':16000},
]
wgt_whatif = make_multiple_forecasts(params, d0, 26)

c1=alt.Chart(wgt_whatif).mark_circle(size=45).encode(
    x=alt.X('date',
        scale=alt.Scale(domain=('2020-01-01', '2020-07-01'))
    ), 
    y=alt.Y('weight',
        scale=alt.Scale(domain=(150, 181))
    ),  
    color='label',
    tooltip=['date', 'weight', 'cals', 'steps']
).properties(
    width=800,
    height=500
).interactive()


c2=alt.Chart(wgt_forecast[wgt_forecast.date>='2020-01-01']).mark_circle(size=60, color='black', opacity=0.4).encode(
    x='date',  
    y=alt.Y('w',
        scale=alt.Scale(domain=(170, 180))
    ),  
    tooltip=['date', 'w', 'w_fc', 'c', 's', 'w_prev']
).properties(
    width=800,
    height=500
).interactive()

c1+c2

alt.LayerChart(...)

In [364]:
d0 = '2020-03-22'
w_prev = 168.5
params = [
    {'wgt_prev':w_prev, 'cals':1900, 'steps':12000},
    {'wgt_prev':w_prev, 'cals':2000, 'steps':12000},
    {'wgt_prev':w_prev, 'cals':1800, 'steps':12000},
    {'wgt_prev':w_prev, 'cals':1900, 'steps':14000},
    {'wgt_prev':w_prev, 'cals':2000, 'steps':14000},
    {'wgt_prev':w_prev, 'cals':1800, 'steps':14000},
    {'wgt_prev':w_prev, 'cals':1900, 'steps':16000},
    {'wgt_prev':w_prev, 'cals':2000, 'steps':16000},
    {'wgt_prev':w_prev, 'cals':1800, 'steps':16000},
]
wgt_whatif = make_multiple_forecasts(params, d0, 15)

c1 = alt.Chart(wgt_whatif).mark_line(size=2).encode(
    x=alt.X('date',
        scale=alt.Scale(domain=(d0, '2020-07-01'))
    ), 
    y=alt.Y('weight',
        scale=alt.Scale(domain=(150, 170))
    ),  
    color='label',
    tooltip=['date', 'weight', 'cals', 'steps']
).properties(
    width=800,
    height=500
).interactive()
c2 = alt.Chart(wgt_whatif).mark_circle(size=45).encode(
    x=alt.X('date',
        scale=alt.Scale(domain=(d0, '2020-07-01'))
    ), 
    y=alt.Y('weight',
        scale=alt.Scale(domain=(150, 170))
    ),  
    color='label',
    tooltip=['date', 'weight', 'cals', 'steps']
).properties(
    width=800,
    height=500
).interactive()

c1+c2

alt.LayerChart(...)

In [349]:
16000*7

112000

In [358]:
d0 = '2020-01-05'
w_prev = 180
params = [
    {'wgt_prev':w_prev, 'cals':2110, 'steps':10000},
]
wgt_whatif = make_multiple_forecasts(params, d0, 4*52)

alt.Chart(wgt_whatif).mark_circle(size=30).encode(
    x=alt.X('date',
        scale=alt.Scale(domain=('2020-01-01', '2023-12-31'))
    ), 
    y=alt.Y('weight',
        scale=alt.Scale(domain=(100, 181))
    ),  
    color='label',
    tooltip=['date', 'weight', 'cals', 'steps']
).properties(
    width=500,
    height=300
).interactive()

alt.Chart(...)

In [361]:
d0 = '2021-01-01'
w_prev = 160
params = [
    {'wgt_prev':w_prev, 'cals':2110, 'steps':10000},
]
wgt_whatif = make_multiple_forecasts(params, d0, 52)

alt.Chart(wgt_whatif).mark_circle(size=30).encode(
    x=alt.X('date',
        scale=alt.Scale(domain=(d0, '2021-12-31'))
    ), 
    y=alt.Y('weight',
        scale=alt.Scale(domain=(100, 181))
    ),  
    color='label',
    tooltip=['date', 'weight', 'cals', 'steps']
).properties(
    width=500,
    height=300
).interactive()

alt.Chart(...)